# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv













Misled by the mitochondrial genome - A phylogenetic study in Topaza hummingbirds	Master’s thesis defense	talk-1	Department of Biological and Environmental Sciences	2015-06-02	"Gothenburg,Sweden"		Master thesis defense presentation - https://www.gu.se/biologi-miljovetenskap/digitalAssets/1536/1536649_tobias-hofmann.pdf

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [4]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Session: Population genetics and phylogeny,Chairing,chair-1,"Evolution in Sweden 2020, Tjärnö Marine Labora...",2020-01-16,"Tjärnö, Sweden",NaN,https://www.gu.se/cemeb-marin-evolutionsbiolog...
1,Estimation of past extinction rates to predict...,Poster,talk-15,"Faculty of Science Poster Conference, Universi...",2019-09-20,"Gothenburg, Sweden",NaN,PhD poster conference
2,Filling gaps in the paleovegetation record - a...,Talk,talk-14,Kew Gardens,2019-06-06,"London, UK",NaN,NaN
3,Estimating diversity and diversification using...,Talk,talk-13,5th International Palaeontological Congress,2018-07-10,"Paris, France",NaN,https://ipc5.sciencesconf.org/
4,Improving DNA sequence data processing for mor...,PhD half-time seminar,talk-12,Department of Biological and Environmental Sci...,2018-06-08,"Gothenburg, Sweden",NaN,PhD half-time seminar https://www.facebook.com...
5,Recent and future extinctions of mammals,Talk,talk-11,"Antonelli, Davis, and Edwards lab meeting, Har...",2018-05-16,"Cambridge, USA",NaN,NaN
6,Estimation of past extinction rates to predict...,Poster,poster_1,20th EGU General Assembly,2018-04-13,"Vienna, Austria",NaN,https://ui.adsabs.harvard.edu/abs/2018EGUGA..2...
7,How much extinction is normal and how severe i...,Talk,talk-10,"ForBio annual meeting, University of Tromsø",2018-02-13,"Tromsø, Norway",NaN,https://www.forbio.uio.no/events/meeting/2018/...
8,The future of Mammalian diversity predicted by...,Talk,talk-9,"Evolution and Systematics meeting, University ...",2017-12-06,"Gothenburg, Sweden",NaN,NaN
9,Mammal extinctions of the recent past,Talk,talk-8,"ForBio annual meeting, University of Bergen",2017-04-25,"Bergen, Norway",NaN,https://www.forbio.uio.no/events/meeting/2017/...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [5]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [9]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [10]:
!ls ../_talks

2015-06-02-talk-1.md   2016-12-08-talk-7.md   2018-06-08-talk-12.md
2015-07-23-talk-2.md   2017-04-25-talk-8.md   2018-07-10-talk-13.md
2015-09-24-talk-3.md   2017-12-06-talk-9.md   2019-06-06-talk-14.md
2016-01-20-talk-4.md   2018-02-13-talk-10.md  2019-09-20-talk-15.md
2016-03-17-talk-5.md   2018-04-13-poster_1.md 2020-01-16-chair-1.md
2016-04-25-talk-6.md   2018-05-16-talk-11.md


In [8]:
!cat ../_talks/2013-03-01-tutorial-1.md

---
title: "Tutorial 1 on Relevant Topic in Your Field"
collection: talks
type: "Tutorial"
permalink: /talks/2013-03-01-tutorial-1
venue: "UC-Berkeley Institute for Testing Science"
date: 2013-03-01
location: "Berkeley CA, USA"
---

[More information here](http://exampleurl.com)

This is a description of your tutorial, note the different field in type. This is a markdown files that can be all markdown-ified like any other post. Yay markdown!
